# Soil HAR 11-62 Classification Recipe - Main Course 🍖

**Purpose**: Process soil data for Hawaii onsite wastewater regulations
**Input**: HIstate_nrcs_join2 (or configurable soil layer)
**Output**: HAR 11-62 classified soil data ready for TMK joining

## Recipe Overview
This is our main dish - the core soil processing that creates Matrix-compatible classifications.

### Ingredients:
- NRCS SSURGO soil data (HIstate_nrcs_join2)
- HAR 11-62 regulatory thresholds
- Matrix compatibility requirements

### Cooking Steps:
1. Load and validate soil data
2. Calculate slope classifications (<8%, 8-12%, >12%)
3. Convert Ksat to percolation rates
4. Apply drainage classifications
5. Calculate Matrix technology compatibility
6. Export processed layer for TMK joining

In [ ]:
# CONFIGURABLE INGREDIENTS
# Change these to swap data sources

# Load master configuration if not already loaded
if 'outputs_folder' not in globals():
    exec(open(r"../00_Master_Menu/00_Master_Hawaii_Matrix_Analysis.ipynb").read())

# Default soil layer (can be overridden from master menu)
if 'soil_layer' not in globals():
    soil_layer = "HIstate_nrcs_join2"

# Output paths
processed_soils_folder = os.path.join(outputs_folder, "Processed_Soils")
if not os.path.exists(processed_soils_folder):
    os.makedirs(processed_soils_folder)

processed_soil_output = f"{soil_layer}_HAR_Processed"
processed_soil_path = os.path.join(processed_soils_folder, processed_soil_output + ".shp")

print(f"🍖 Cooking with soil layer: {soil_layer}")
print(f"📤 Output will be: {processed_soil_path}")

import sys
sys.path.append(os.path.join(scripts_folder, '99_Kitchen_Utils'))
from Common_Functions import *

In [ ]:
# STEP 1: VALIDATE AND PREPARE SOIL DATA
log_workflow_step("Soil Processing", "Starting HAR 11-62 classification")

# Validate source layer
validate_layer_exists(soil_layer)

# Create working copy
arcpy.management.CopyFeatures(soil_layer, processed_soil_path)
print(f"✅ Created working copy: {processed_soil_path}")

# Verify required fields
required_fields = ['slope_r', 'ksat_r', 'drainagecl', 'musym']
existing_fields = [f.name for f in arcpy.ListFields(processed_soil_path)]
missing_fields = [f for f in required_fields if f not in existing_fields]

if missing_fields:
    raise ValueError(f"Missing required fields: {missing_fields}")
else:
    print(f"✅ All required fields present: {required_fields}")

# Get record count
record_count = int(arcpy.management.GetCount(processed_soil_path)[0])
print(f"Processing {record_count:,} soil records")

In [ ]:
# STEP 2: ADD HAR 11-62 CLASSIFICATION FIELDS
log_workflow_step("Soil Processing", "Adding HAR 11-62 fields")

# Define new fields for HAR 11-62 analysis
har_fields = [
    ("HAR_SLOPE_CLASS", "TEXT", 15, "HAR 11-62 Slope Classification"),
    ("HAR_PERC_CLASS", "TEXT", 20, "HAR 11-62 Percolation Classification"), 
    ("HAR_DRAINAGE_CLASS", "TEXT", 20, "Drainage Suitability for Septic"),
    ("PERC_RATE_EST", "DOUBLE", None, "Estimated Percolation Rate (min/inch)"),
    ("MATRIX_SEPTIC_OK", "SHORT", None, "Standard Septic Compatible (1/0)"),
    ("MATRIX_ATU_OK", "SHORT", None, "ATU System Compatible (1/0)"),
    ("MATRIX_SEEPAGE_PIT_OK", "SHORT", None, "Seepage Pit Compatible (1/0)"),
    ("LIMITING_FACTORS", "TEXT", 100, "What limits technology use")
]

# Add fields
for field_name, field_type, field_length, field_alias in har_fields:
    try:
        if field_length:
            arcpy.management.AddField(processed_soil_path, field_name, field_type, 
                                   field_length=field_length, field_alias=field_alias)
        else:
            arcpy.management.AddField(processed_soil_path, field_name, field_type, 
                                   field_alias=field_alias)
        print(f"  ✅ Added: {field_name}")
    except Exception as e:
        print(f"  ⚠️  Field {field_name}: {e}")

In [ ]:
# STEP 3: CALCULATE SLOPE CLASSIFICATIONS
log_workflow_step("Soil Processing", "Calculating slope classes")

slope_counts = {"<8%": 0, "8-12%": 0, ">12%": 0, "Unknown": 0}

with arcpy.da.UpdateCursor(processed_soil_path, ["slope_r", "HAR_SLOPE_CLASS"]) as cursor:
    for row in cursor:
        if row[0] is not None:
            slope = float(row[0])
            if slope < 8:
                row[1] = "<8%"
                slope_counts["<8%"] += 1
            elif slope <= 12:
                row[1] = "8-12%"
                slope_counts["8-12%"] += 1
            else:
                row[1] = ">12%"
                slope_counts[">12%"] += 1
        else:
            row[1] = "Unknown"
            slope_counts["Unknown"] += 1
        cursor.updateRow(row)

print("Slope Distribution:")
for slope_class, count in slope_counts.items():
    percentage = (count / record_count) * 100
    print(f"  {slope_class}: {count:,} records ({percentage:.1f}%)")

In [ ]:
# STEP 4: CONVERT KSAT TO PERCOLATION RATES
log_workflow_step("Soil Processing", "Converting Ksat to percolation rates")

perc_counts = {"<1 min/inch": 0, "1-10 min/inch": 0, "10-60 min/inch": 0, ">60 min/inch": 0, "Unknown": 0}

with arcpy.da.UpdateCursor(processed_soil_path, ["ksat_r", "PERC_RATE_EST", "HAR_PERC_CLASS"]) as cursor:
    processed_count = 0
    for row in cursor:
        if row[0] is not None and row[0] > 0:
            # Convert micrometers/second to minutes/inch
            # Formula: perc_rate = 4233.3 / ksat_micrometers_per_sec
            ksat = float(row[0])
            perc_rate = 4233.3 / ksat
            row[1] = round(perc_rate, 2)
            processed_count += 1
            
            # HAR 11-62 percolation classifications
            if perc_rate < 1:
                row[2] = "<1 min/inch"
                perc_counts["<1 min/inch"] += 1
            elif perc_rate <= 10:
                row[2] = "1-10 min/inch"
                perc_counts["1-10 min/inch"] += 1
            elif perc_rate <= 60:
                row[2] = "10-60 min/inch"
                perc_counts["10-60 min/inch"] += 1
            else:
                row[2] = ">60 min/inch"
                perc_counts[">60 min/inch"] += 1
        else:
            row[1] = None
            row[2] = "Unknown"
            perc_counts["Unknown"] += 1
        cursor.updateRow(row)

print(f"Processed {processed_count:,} records with valid Ksat values")
print("Percolation Rate Distribution:")
for perc_class, count in perc_counts.items():
    percentage = (count / record_count) * 100
    print(f"  {perc_class}: {count:,} records ({percentage:.1f}%)")

In [ ]:
# STEP 5: CLASSIFY DRAINAGE SUITABILITY
log_workflow_step("Soil Processing", "Classifying drainage suitability")

drainage_lookup = {
    "Very limited": "Poor",
    "Somewhat limited": "Moderate", 
    "Not rated": "Unknown",
    "Well drained": "Good",
    "Moderately well drained": "Good",
    "Somewhat poorly drained": "Moderate",
    "Poorly drained": "Poor",
    "Very poorly drained": "Poor",
    "Excessively drained": "Good"
}

drainage_counts = {"Good": 0, "Moderate": 0, "Poor": 0, "Unknown": 0}

with arcpy.da.UpdateCursor(processed_soil_path, ["drainagecl", "HAR_DRAINAGE_CLASS"]) as cursor:
    for row in cursor:
        drainage = row[0] if row[0] else "Unknown"
        classified = drainage_lookup.get(drainage, "Unknown")
        row[1] = classified
        drainage_counts[classified] += 1
        cursor.updateRow(row)

print("Drainage Distribution:")
for drainage_class, count in drainage_counts.items():
    percentage = (count / record_count) * 100
    print(f"  {drainage_class}: {count:,} records ({percentage:.1f}%)")

In [ ]:
# STEP 6: CALCULATE MATRIX TECHNOLOGY COMPATIBILITY
log_workflow_step("Soil Processing", "Calculating Matrix compatibility")

compatibility_counts = {
    "SEPTIC": {"Compatible": 0, "Not Compatible": 0},
    "ATU": {"Compatible": 0, "Not Compatible": 0},
    "SEEPAGE_PIT": {"Compatible": 0, "Not Compatible": 0}
}

fields = ["HAR_SLOPE_CLASS", "HAR_PERC_CLASS", "HAR_DRAINAGE_CLASS",
          "MATRIX_SEPTIC_OK", "MATRIX_ATU_OK", "MATRIX_SEEPAGE_PIT_OK", 
          "LIMITING_FACTORS"]

with arcpy.da.UpdateCursor(processed_soil_path, fields) as cursor:
    for row in cursor:
        slope_class = row[0] or "Unknown"
        perc_class = row[1] or "Unknown"
        drainage_class = row[2] or "Unknown"
        
        limitations = []
        
        # STANDARD SEPTIC COMPATIBILITY
        septic_ok = (
            slope_class in ["<8%", "8-12%"] and
            perc_class in ["1-10 min/inch", "10-60 min/inch"] and
            drainage_class in ["Good", "Moderate"]
        )
        row[3] = 1 if septic_ok else 0
        compatibility_counts["SEPTIC"]["Compatible" if septic_ok else "Not Compatible"] += 1
        
        if not septic_ok:
            if slope_class == ">12%":
                limitations.append("Steep slope")
            if perc_class in ["<1 min/inch", ">60 min/inch"]:
                limitations.append("Poor percolation")
            if drainage_class == "Poor":
                limitations.append("Poor drainage")
        
        # ATU COMPATIBILITY (more flexible)
        atu_ok = (
            slope_class in ["<8%", "8-12%"] and
            perc_class in ["<1 min/inch", "1-10 min/inch", "10-60 min/inch"] and
            drainage_class in ["Good", "Moderate"]
        )
        row[4] = 1 if atu_ok else 0
        compatibility_counts["ATU"]["Compatible" if atu_ok else "Not Compatible"] += 1
        
        # SEEPAGE PIT COMPATIBILITY (steep slopes)
        seepage_pit_ok = (
            slope_class == ">12%" and
            perc_class == "1-10 min/inch" and
            drainage_class in ["Good", "Moderate"]
        )
        row[5] = 1 if seepage_pit_ok else 0
        compatibility_counts["SEEPAGE_PIT"]["Compatible" if seepage_pit_ok else "Not Compatible"] += 1
        
        # Record limiting factors
        row[6] = "; ".join(limitations) if limitations else "Suitable"
        
        cursor.updateRow(row)

print("\n=== MATRIX COMPATIBILITY RESULTS ===")
for tech, counts in compatibility_counts.items():
    total = counts["Compatible"] + counts["Not Compatible"]
    compatible_pct = (counts["Compatible"] / total) * 100
    print(f"{tech}:")
    print(f"  Compatible: {counts['Compatible']:,} ({compatible_pct:.1f}%)")
    print(f"  Not Compatible: {counts['Not Compatible']:,} ({100-compatible_pct:.1f}%)")

In [ ]:
# STEP 7: FINALIZE AND DOCUMENT
log_workflow_step("Soil Processing", "Finalizing processed soil layer")

# Add to map for verification
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_obj = aprx.activeMap
map_obj.addDataFromPath(processed_soil_path)

print(f"\n✅ SOIL PROCESSING COMPLETE!")
print(f"📁 Output: {processed_soil_path}")
print(f"📊 Records processed: {record_count:,}")
print(f"🗺️ Added to map for verification")
print(f"\n🍽️ Ready for TMK joining in next course!")

# Save processing summary
summary_path = os.path.join(processed_soils_folder, f"{processed_soil_output}_SUMMARY.txt")
with open(summary_path, 'w') as f:
    f.write(f"SOIL HAR 11-62 PROCESSING SUMMARY\n")
    f.write(f"Processed: {datetime.now()}\n")
    f.write(f"Source: {soil_layer}\n")
    f.write(f"Output: {processed_soil_output}\n")
    f.write(f"Records: {record_count:,}\n\n")
    
    f.write("SLOPE DISTRIBUTION:\n")
    for slope, count in slope_counts.items():
        f.write(f"  {slope}: {count:,}\n")
    
    f.write("\nCOMPATIBILITY RESULTS:\n")
    for tech, counts in compatibility_counts.items():
        f.write(f"  {tech}: {counts['Compatible']:,} compatible\n")

log_workflow_step("Soil Processing", "HAR 11-62 classification complete")